# Python Problem: python script which can fetch all the tweets done by ​midas@IIITD​ twitter handle and dump the responses into JSONlines file.

#### Importing the libraries
 
 - json: for saving and loading of data
 - requests: for sending GET request to Twitter API
 - requests_oauthlib: for authentication on Twitter API
 - BeautifulSoup for webScrapping

In [4]:
import sys
import json
import requests
from requests_oauthlib import OAuth1
from bs4 import BeautifulSoup

#### Function to authenticate twitter user API

In [3]:
def twi_auth():

    CONSUMER_KEY = "KIrd3dd7BSB9UT5DNa15y2DG9"
    CONSUMER_SECRET = "YDifo1C6RmIqP9aOlR9Mi83jtCm2AbXH099vCHz2ERZKgOz50s"
    ACCESS_KEY = "1058038441877753856-WpIEfvaiznbh0FTkntAdpAJ3i1lR53"
    ACCESS_SECRET = "VjUZI1jQYoHsRzovInyXL1tpyBg2NPIYID4WSX2MmCNZ8"

    auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, ACCESS_KEY, ACCESS_SECRET)

    return auth

#### Function to get tweet count

 - Used Web Scrapping  to get total tweets from a particular twitter handle
 - Returns the tweet count

In [ ]:
def twi_count(twi_handle):
    
    temp = requests.get('https://twitter.com/'+twi_handle)
    bs = BeautifulSoup(temp.text,'lxml')
    count=0
    
    try:
        tweet_box = bs.find('li',{'class':'ProfileNav-item ProfileNav-item--tweets is-active'})
        tweets= tweet_box.find('a').find('span',{'class':'ProfileNav-value'})
        count=tweets.get('data-count')

    except:
        print('Account name not found...')
        sys.exit(1)
    
    return count
        